In [1]:
import numpy as np
from neuron import h
h.nrn_load_dll('biophysical_mods/nrnmech.dll')

1.0

In [2]:
soma_skeleton=np.zeros([3,100])
soma_skeleton[0,:]=np.linspace(1,1000,100)-500
soma_skeleton[1,:]=np.linspace(1,1000,100)-500
soma_skeleton[2,:]=np.linspace(1,1000,100)-500
diam=np.linspace(2,20,100)

soma = h.Section(name='soma')
soma.nseg = np.shape(soma_skeleton)[1]

soma.insert('pas')
soma.Ra = 1000
soma.cm=1
soma.e_pas=-90
soma.insert('Ca_LVAst')
soma.insert('Ca_HVA')
soma.insert('SKv3_1')
soma.insert('SK_E2')
soma.insert('K_Tst')
soma.insert('K_Pst')
soma.insert('Nap_Et2')
soma.insert('NaTa_t')
soma.insert('CaDynamics_E2')
soma.insert('Ih')

h.pt3dadd(h.Vector(soma_skeleton[0,:]), h.Vector(soma_skeleton[1,:]), h.Vector(soma_skeleton[2,:]),h.Vector(diam), sec=soma)

connections=np.zeros([len(diam)-1,2])
connections[:,0]=np.linspace(1,len(diam)-1,len(diam)-1)
connections[:,1]=np.linspace(0,len(diam)-2,len(diam)-1)

for i,seg in enumerate(soma):
    seg.ek = -85
    seg.ena = 50
    seg.gIhbar_Ih = 0.0002
    seg.g_pas = 0.0000344
    seg.decay_CaDynamics_E2 = 486.0
    seg.gamma_CaDynamics_E2 = 0.000549
    seg.gCa_LVAstbar_Ca_LVAst = 0.00432
    seg.gCa_HVAbar_Ca_HVA = 0.000567
    seg.gSKv3_1bar_SKv3_1 = 0.766
    seg.gSK_E2bar_SK_E2 = 0.0556
    seg.gK_Tstbar_K_Tst = 0.0326
    seg.gK_Pstbar_K_Pst = 0.000547
    seg.gNap_Et2bar_Nap_Et2 = 0.00496
    seg.gNaTa_tbar_NaTa_t = 1.71

stim = h.epsp(soma(0.5))
stim.onset = 0.2
stim.imax = 5
#stim.dur = 0.3
h.load_file('stdrun.hoc')
h.finitialize(-70)
h.dt=0.05


In [3]:
from mayavi import mlab
mlab.options.offscreen = False
mlab.figure(size=(1000,700))
points = mlab.pipeline.scalar_scatter(soma_skeleton[0,:], soma_skeleton[1,:], soma_skeleton[2,:], diam/2.0)
dataset = points.mlab_source.dataset
dataset.point_data.get_array(0).name = 'diameter'
dataset.lines = connections
dataset.point_data.update()
# The tube
src = mlab.pipeline.set_active_attribute(points, point_scalars='diameter')
stripper = mlab.pipeline.stripper(src)


tube = mlab.pipeline.tube(stripper, tube_sides = 6, tube_radius = 1)
tube.filter.capping = True
#        tube.filter.use_default_normal = False
tube.filter.vary_radius = 'vary_radius_by_absolute_scalar'

tube.children = [] # Removing the old ones 

rest_potential = np.linspace(-85,25,len(diam))
array_id = dataset.point_data.add_array(rest_potential)
dataset.point_data.get_array(array_id).name = 'voltage'
dataset.point_data.update()
src2 = mlab.pipeline.set_active_attribute(tube, 
                                            point_scalars='voltage')
surf = mlab.pipeline.surface(src2, colormap='gist_heat')

cb=mlab.colorbar(object=surf,nb_labels=10)
lut=surf.module_manager.scalar_lut_manager.lut.table.to_array()
'''
lut = np.zeros([256,4])

lut[0:90,1]=np.zeros(90)
lut[90:180,1]=np.linspace(1,90,90)
lut[180:,1]=np.zeros(76)+90

lut[0:180,0]=np.zeros(180)
lut[180:,0]=np.linspace(1,76,76)

lut[0:90,2]=np.linspace(0,90,90)
lut[90:,2]=np.zeros(166)+90
'''

lut[:,3]=np.linspace(0,255,256)
lut[0:10,3]=np.zeros(10)+10

surf.module_manager.scalar_lut_manager.lut.table = lut
surf.module_manager.scalar_lut_manager.use_default_range=False
membrane_potential=np.zeros(len(diam))
@mlab.animate(delay=100)
def anim():
    surf.actor.actor.rotate_z(90)
    #f = mlab.gcf()
    for i in range(50):
        h.fadvance()
        for j,seg in enumerate(soma):
            membrane_potential[j]=seg.v
        
        dataset.point_data.remove_array(1)
        dataset.point_data.update()
        
        # Adding the new scalar
        dataset.point_data.add_array(membrane_potential)
        dataset.point_data.get_array(1).name = 'voltage'
        dataset.point_data.update()
        #mlab.draw()
        
        # Updating the dataset
        #ms = surf.mlab_source
        #ms.m_data.update()

        #surf.mlab_source.voltage = np.ones(10)*i*200
        #f.scene.render()
        mlab.process_ui_events()
        yield
        print(i,h.t)
        mlab.savefig('video/image'+str(i)+'.png')

anim()
mlab.show()


0 0.05
1 0.1
2 0.15
3 0.19999999999999998
4 0.24999999999999997
5 0.3
6 0.35000000000000003
7 0.4000000000000001
8 0.4500000000000001
9 0.5000000000000001
10 0.5500000000000002
11 0.6000000000000002
12 0.6500000000000002
13 0.7000000000000003
14 0.7500000000000003
15 0.8000000000000004
16 0.8500000000000004
17 0.9000000000000005
18 0.9500000000000005
19 1.0000000000000004
20 1.0500000000000003
21 1.1
22 1.15
23 1.1999999999999997
24 1.2499999999999996
25 1.2999999999999994
26 1.3499999999999992
27 1.399999999999999
28 1.4499999999999988
29 1.4999999999999987
30 1.5499999999999985
31 1.5999999999999983
32 1.6499999999999981
33 1.699999999999998
34 1.7499999999999978
35 1.7999999999999976
36 1.8499999999999974
37 1.8999999999999972
38 1.949999999999997
39 1.999999999999997
40 2.0499999999999967
41 2.0999999999999965
42 2.1499999999999964
43 2.199999999999996
44 2.249999999999996
45 2.299999999999996
46 2.3499999999999956
47 2.3999999999999955
48 2.4499999999999953
49 2.499999999999995


In [4]:
from mayavi import mlab
mlab.options.offscreen = False
mlab.figure(size=(1000,700))

h.load_file('stdrun.hoc')
h.finitialize(-75)
h.dt=0.05

print(len(soma_skeleton),np.shape(soma_skeleton))
soma_skeleton[0,:] = soma_skeleton[0,:] - np.median(soma_skeleton[0,:])
soma_skeleton[1,:] = soma_skeleton[1,:] - np.median(soma_skeleton[1,:])
soma_skeleton[2,:] = soma_skeleton[2,:] - np.median(soma_skeleton[2,:])
points = mlab.pipeline.scalar_scatter(soma_skeleton[0,:], soma_skeleton[1,:], soma_skeleton[2,:], diam)
dataset = points.mlab_source.dataset
dataset.point_data.get_array(0).name = 'diameter'
dataset.lines = connections
dataset.point_data.update()

# The tube
src = mlab.pipeline.set_active_attribute(points, point_scalars='diameter')
stripper = mlab.pipeline.stripper(src)
tube = mlab.pipeline.tube(stripper, tube_sides = 6, tube_radius = 1)
tube.filter.capping = True
#        tube.filter.use_default_normal = False
tube.filter.vary_radius = 'vary_radius_by_absolute_scalar'
tube.children = [] # Removing the old ones 

rest_potential = np.linspace(-75,20,len(diam))
array_id = dataset.point_data.add_array(rest_potential)
dataset.point_data.get_array(array_id).name = 'voltage'
dataset.point_data.update()
src2 = mlab.pipeline.set_active_attribute(tube, 
                                            point_scalars='voltage')
surf = mlab.pipeline.surface(src2, colormap='gist_heat')

cb=mlab.colorbar(object=surf,nb_labels=10)
lut=surf.module_manager.scalar_lut_manager.lut.table.to_array()
lut[:,3]=np.linspace(0,255,256)
lut[0:10,3]=np.zeros(10)+10

surf.module_manager.scalar_lut_manager.lut.table = lut
surf.module_manager.scalar_lut_manager.use_default_range=False
membrane_potential = np.zeros(len(diam))

@mlab.animate(delay=100)
def anim():
    surf.actor.actor.rotate_y(40)
    surf.actor.actor.rotate_x(-30)
    surf.actor.actor.rotate_z(30)
    #surf.actor.actor.rotate_y(45)
    #f = mlab.gcf()

    for i in range(50):
        
        h.fadvance()
        i_seg = 0
        N = soma.n3d()
        #print(i,h.t)
        for seg in range(soma.n3d()):
            membrane_potential[i_seg]=soma(soma.arc3d(seg)/soma.L).v
            i_seg = i_seg + 1
            print(soma.arc3d(seg))
            '''
                    for i_sec in section_index:
            N=N+dend[i_sec].n3d()
            for seg in range(dend[i_sec].n3d()):
                membrane_potential[i_seg]=dend[i_sec](dend[i_sec].arc3d(seg)/dend[i_sec].L).v
                i_seg = i_seg + 1
            '''

        print(N,i_seg)
        dataset.point_data.remove_array(1)
        dataset.point_data.update()
        
        # Adding the new scalar
        dataset.point_data.add_array(membrane_potential)
        dataset.point_data.get_array(1).name = 'voltage'
        dataset.point_data.update()
        #mlab.draw()
        
        # Updating the dataset
        #ms = surf.mlab_source
        #ms.m_data.update()

        #surf.mlab_source.voltage = np.ones(10)*i*200
        #f.scene.render()
        mlab.process_ui_events()
        yield
        mlab.savefig('video/image'+str(i)+'.png')
anim()
mlab.show()


3 (3, 100)
0.0
17.477972045285565
34.95594409057113
52.43391613585669
69.91188818114226
87.38986022642783
104.86777941371756
122.34575145900313
139.8237235042887
157.30169554957425
174.7796675948598
192.25763964014536
209.7356116854309
227.21358373071646
244.69155577600202
262.1695278212876
279.6474998665732
297.1254190538629
314.6033910991485
332.0813631444341
349.55933518971966
367.03730723500524
384.5152792802908
401.9932513255764
419.471223370862
436.9491689871496
454.4271410324352
471.9051130777208
489.38308512300637
506.86105716829195
524.3390292135775
541.8169748298652
559.2949468751508
576.7729189204364
594.250890965722
611.7288630110075
629.2068086272952
646.6847938870798
664.1627527178664
681.640724763152
699.1186835939386
716.5966556392242
734.0746276845098
751.5525865152964
769.030558560582
786.5085239986181
803.9864960439037
821.4644614819398
838.9424269199759
856.4203956616367
873.8983627514851
891.3763314931459
908.854296931182
926.3322623692181
943.8102344145037
961.288